In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的AutoML文本分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/>

## 概述

本教程演示如何使用 Python 的 Vertex 客户端库来创建文本分类模型，并使用 Google Cloud 的 [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 进行在线预测。

数据集

本教程使用的数据集是来自Kaggle数据集的[Happy Moments dataset](https://www.kaggle.com/ritresearch/happydb)。在本教程中使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML文本分类模型，并部署用于在线预测。您也可以选择使用`gcloud`命令行工具或Google Cloud Console在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 部署`Model`资源到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的 Vertex 客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦你安装了Vertex客户端库和Google *cloud-storage*，你需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有此选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google 云项目

**无论您使用何种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您会获得 $300 的免费信用，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已安装在 Google Cloud 笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 开头的行作为 shell 命令运行，并将以 `$` 开头的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改用于此笔记本其余部分操作的 `REGION` 变量。以下是 Vertex 支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来进行 Vertex 训练。并非所有地区都支持所有 Vertex 服务。有关每个地区的最新支持，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间资源名称的冲突，您为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐号**。

在**服务帐号名称**字段中，输入一个名称，然后单击**创建**。

在**将此服务帐号授予项目访问权限**部分，单击角色下拉列表。在筛选框中键入"Vertex"，然后选择**Vertex管理员**。在筛选框中键入"Storage Object Admin"，然后选择**Storage Object Admin**。

单击创建。包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐户密钥的路径输入作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

设置变量

接下来设置一些在教程中使用的变量。
导入库并定义常量。

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、管道和终端服务端点。
- `PARENT`：顶点资源的数据集、模型、作业、管道和终端资源的根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

自动机器学习常量

设置独特于AutoML数据集和训练的常量：

- 数据集模式：告诉“数据集”资源服务数据集的类型。
- 数据标注（注释）模式：告诉“数据集”资源服务数据如何被标记（注释）。
- 数据集训练模式：告诉“管道”资源服务为哪种任务（例如分类）训练模型。

In [ ]:
# Text Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
# Text Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_classification_single_label_io_format_1.0.0.yaml"
# Text Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_classification_1.0.0.yaml"

# 教程

现在你已经准备好开始创建自己的AutoML文本分类模型了。

## 设置客户端

Vertex 客户端库以客户端/服务器模型运行。在您的端（Python 脚本）中，您将创建一个客户端，用于向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于 `Dataset` 资源的 Dataset 服务。
- 用于 `Model` 资源的 Model 服务。
- 用于训练的 Pipeline 服务。
- 用于部署的 Endpoint 服务。
- 用于提供预测的 Prediction 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户已经准备好，训练模型的第一步是创建一个受管数据集实例，然后将带有标签的数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
   - `display_name`：您选择指定的人类可读名称。
   - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
   - `parent`：您的`数据库`，`模型`和`端点`资源的Vertex位置根路径。
   - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用方式。虽然这一步通常很快，但当您首次在项目中使用它时，由于规定，可能会有较长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，方法是调用操作方法：

| 方法          | 描述               |
| ----------- | ----------------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。 |
| running()   | 返回操作是否仍在运行的True/False。       |
| done()      | 返回操作是否已完成的True/False。           |
| canceled()  | 返回操作是否已取消的True/False。         |
| cancel()    | 取消操作（可能需要最多30秒）。             |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("happydb-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

顶点 `Dataset` 文本资源对您的文本数据有一些要求。

- 文本示例必须存储在 CSV 或 JSONL 文件中。

CSV文件

对于文本分类，CSV文件有几个要求：

- 没有标题。
- 第一列是文本示例或文本文件的云存储路径（.txt后缀）。
- 第二列是标签。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数 (`wc -l`) 来确定示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex Dataset 资源中。使用这个辅助函数 `import_data` 来导入数据。该函数会执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并使用以下参数：
 - `name`：您为 `Dataset` 资源提供的易读名称（例如，happydb）。
 - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
 - `gcs_sources`：一个包含一个或多个索引文件路径的 URI 列表。
 - `import_schema_uri`：标识标签类型的模式。

`import_data()` 方法会返回一个长时间运行的 `operation` 对象。这将需要几分钟的时间才能完成。如果您正在进行实时教程，现在是问问题或休息一下的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在使用您的Vertex `Dataset`资源训练一个AutoML文本分类模型。要训练模型，请按以下步骤操作：

1. 为`Dataset`资源创建一个Vertex训练流水线。
2. 执行流水线开始训练。

### 创建一个训练流水线

您可能会问，我们用流水线做什么？通常在工作（例如训练）有多个步骤且按顺序进行时，我们会使用流水线：执行步骤A，执行步骤B，等等。将这些步骤放入流水线中，我们可以获得以下好处：

1. 可以重复用于后续的训练工作。
2. 可以容器化并作为批处理工作运行。
3. 可以分布式执行。
4. 所有步骤与同一流水线工作相关联，以便跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：流水线工作的一个易读名称。
- `model_name`：模型的一个易读名称。
- `dataset`：Vertex 完全合格的数据集标识符。
- `schema`：数据集标注（注释）训练模式。
- `task`：描述训练工作要求的字典。

辅助函数调用了 `Pipeline` 客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：您的 `Dataset`，`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线训练工作的完整规范。

现在让我们更深入地看一下构建 `training_pipeline` 规范所需的 *最低* 要求：

- `display_name`：流水线工作的一个易读名称。
- `training_task_definition`：数据集标注（注释）训练模式。
- `training_task_inputs`：描述训练工作要求的字典。
- `model_to_upload`：模型的一个易读名称。
- `input_data_config`：数据集规范。
  - `dataset_id`：仅 Vertex 数据集标识符（非完全合格）--这是完全合格标识符的最后部分。
  - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，其他参数需要Python（类似JSON格式）字典，`task`字段则需要一个Google protobuf Struct，它非常类似于Python字典。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段是：

- `multi_label`：指定是否是多标签（而不是单一）分类。

最后，通过调用帮助函数`create_pipeline`来创建管道，该函数返回一个训练管道对象的实例。

In [ ]:
PIPE_NAME = "happydb_pipe-" + TIMESTAMP
MODEL_NAME = "happydb_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "multi_label": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取培训管道的信息

现在仅获取此培训管道实例的管道信息。通过调用作业客户服务的`get_training_pipeline`方法，使用以下参数来获取此作业的作业信息的帮助函数：

- `name`：Vertex的完全限定管道标识符。

当模型训练完成时，管道的状态将是`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练上述模型可能需要超过120分钟的时间。

一旦您的模型训练完成，您可以通过将'end_time'减去'start_time'来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的Vertex模型资源标识符，即流水线服务分配给它的。您可以从返回的流水线实例中获取此信息，作为字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为你的模型多好。在训练过程中，数据集的一部分被保留为测试（留存）数据，由管道服务用于评估模型。

### 列出所有切片的评估

使用这个帮助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的Vertex全限定模型标识符。

该帮助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），我们将打印出评估中每个指标的所有键名，并对于一个小集合（`logLoss`和`auPrc`），您将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 部署 `Model` 资源

现在部署你用AutoML创建的训练好的Vertex `Model` 资源。 这需要两个步骤：

1. 为 `Model` 资源创建一个用于部署的 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

### 创建一个`Endpoint`资源

使用这个辅助函数 `create_endpoint` 来创建一个端点，用于部署模型以提供预测，参数如下：

- `display_name`: 为`Endpoint`资源指定一个易于理解的名称。

这个辅助函数使用端点客户端服务的 `create_endpoint` 方法来创建端点资源，该方法接受以下参数：

- `display_name`: 为`Endpoint`资源指定一个易于理解的名称。

创建一个`Endpoint`资源将返回一个长时间运行的操作，因为为提供服务而准备`Endpoint`资源可能需要一些时间。您可以调用 `response.result()`，这是一个同步调用，当`Endpoint`资源准备就绪时会返回。这个辅助函数返回`Endpoint`资源的Vertex完全合格标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "happydb_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理您的在线预测请求时，您有几种计算实例扩展的选择：

- 单个实例：在线预测请求在单个计算实例上处理。
   - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一个。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例之间分配。
   - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例将被预配，并且在线预测请求将均匀分布在它们之间。

- 自动扩展：在线预测请求在可扩展的计算实例数量之间分配。
   - 将计算实例的最小（`MIN_NODES`）数量设置为在模型首次部署时进行预配和取消预配，并根据负载情况设置计算实例的最大（`MAX_NODES`）数量进行预配。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您之后的部署请求中使用。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将从训练管道中上传（部署）的 `Model` 资源部署到为提供预测而创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 要将模型部署到的 Vertex 完全限定终端标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 到达终端的流量百分比，这是指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 是指上传的这个模型，100 意味着100% 的流量。
   - 如果终端上有现有模型，将分配流量，则使用 `model_id` 指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是要部署到的现有模型的模型标识符。百分比必须加起来为100。

现在让我们更深入地了解 `deployed_model` 参数。该参数被指定为一个带有最小必需字段的 Python 字典：

- `model`: 要部署的（上传）模型的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 这将禁用容器事件的记录，例如执行故障（默认情况下容器日志记录是启用的）。在调试部署时通常启用容器日志记录，然后在生产部署时禁用。
- `automatic_resources`: 这指的是多少个冗余计算实例（副本）。在这个示例中，我们将其设置为一个（不复制）。

#### 流量分配

现在让我们更深入地了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。起初这可能有点令人困惑。让我解释一下，您可以将您的模型的多个实例部署到一个终端，并设置每个实例接收多少（百分比）的流量。

为什么要这样做？也许您已经在生产环境中部署了上一个版本 -- 让我们称其为 v1。在 v2 上进行了更好的模型评估，但在您部署到生产环境之前，您并不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的终端，但只接收到10% 的流量。这样，您可以监视其运行情况，而不会干扰大多数用户 -- 直到您做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将同步等待操作完成，通过调用 `response.result()`，该方法将阻塞直到模型被部署。如果这是第一次将模型部署到终端，则可能需要几分钟来完成资源的分配。

In [ ]:
DEPLOYED_NAME = "happydb_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将从数据集中选择一个任意的示例作为测试项目。不用担心该示例可能已经在训练模型中使用过 -- 我们只是想演示如何进行预测。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您有一个测试项目。使用这个辅助函数`predict_item`，它接受以下参数：

- `filename`：测试项目的云存储路径。
- `endpoint`：用于部署`Model`资源的`Endpoint`资源的 Vertex 完全限定标识符。
- `parameters_dict`：用于提供预测结果的附加过滤参数。

这个函数调用预测客户端服务的`predict`方法，参数如下：

- `endpoint`：用于部署`Model`资源的`Endpoint`资源的 Vertex 完全限定标识符。
- `instances`：要预测的实例（文本文件）列表。
- `parameters`：提供预测结果的附加过滤参数。*注*，文本模型不支持附加参数。

#### 请求

每个实例的格式为：

    { 'content': text_item }

因为`predict()`方法可以接受多个项目（实例），所以您将单个测试项目发送为一个测试项目列表。最后一步是将实例列表打包成谷歌的协议缓冲区格式 -- 这就是传递给`predict()`方法的内容。

#### 响应

`response`对象返回一个列表，其中列表中的每个元素对应请求中的相应文本。您会在每个预测的输出中看到：

- `confidences`：预测的置信水平。
- `displayNames`：预测的标签。

In [ ]:
def predict_item(data, endpoint, parameters_dict):

    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{"content": data}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))
    return response


response = predict_item(test_item, endpoint_id, None)

## 取消部署`Model`资源

现在在服务型`Endpoint`资源上取消部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将`Model`资源部署到服务端点时，端点服务返回的模型部署标识符。
- `endpoint`：被部署到的`Model`的Vertex完全限定标识符的`Endpoint`资源。

这个函数调用端点客户端服务的`undeploy_model`方法，并使用以下参数：

- `deployed_model_id`：在将`Model`资源部署时端点服务返回的模型部署标识符。
- `endpoint`：被部署的`Model`资源位于的`Endpoint`资源的Vertex完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上的其余部署模型间分配流量。

由于这是`Endpoint`资源上唯一的已部署模型，您可以通过设置`traffic_split`为空（即设置为{}）来简单地保持不变。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

为了清理这个项目中使用的所有GCP资源，您可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME